# SE3-TrackNet
## Import

In [2]:
import numpy as np
from predict import Tracker, project_points
from Utils import *
import yaml
import os
import cv2
from data_augmentation import *
from se3_tracknet import *
from datasets import *

import open3d as o3d
import sys,shutil
import os
from datasets import *
from multiprocessing import cpu_count
import argparse
from problems import *
from network_modules import *
from se3_tracknet import *
import torch
from torch import optim
import torch.utils.data
import numpy as np
import yaml
import glob
import random
from data_augmentation import *

os.environ['DISPLAY'] = ':1'

## Init model

In [2]:
path_train_output = f"/home/se3_tracknet/train_output2"
path_train_output = f"/home/se3_tracknet/train_output"

file_dataset_info = f"{path_train_output}/dataset_info.yml"
file_images_mean = f"{path_train_output}/mean.npy"
file_images_std = f"{path_train_output}/std.npy"
file_ckpt_dir = f"{path_train_output}/model_best_val.pth.tar"
trans_normalizer = 0.02
rot_normalizer = 15

with open(file_dataset_info,'r') as ff:
    dataset_info = yaml.safe_load(ff)

images_std = np.load(file_images_std)
images_mean = np.load(file_images_mean)
tracker = Tracker(dataset_info,images_mean,images_std,file_ckpt_dir,model_path=dataset_info['models'][0]['model_path'],trans_normalizer=trans_normalizer,rot_normalizer=rot_normalizer)

self.object_cloud loaded and downsampled
self.object_width= 148.76080761074135
Loading ckpt from  /home/se3_tracknet/train_output/model_best_val.pth.tar
vorher_1:
Allocated: 52.55322265625 MB
Cached: 62.0 MB
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   53814 KB |   53814 KB |   53814 KB |       0 B  |
|       from large pool |   52736 KB |   52736 KB |   52736 KB |       0 B  |
|       from small pool |    1078 KB |    1078 KB |    1078 KB |       0 B  |
|-----------

ValueError: Failed to initialize Pyglet window with an OpenGL >= 3+ context. If you're logged in via SSH, ensure that you're running your script with vglrun (i.e. VirtualGL). The internal error message was "Cannot connect to "None""

## Test sample with visualization
direkter RGB und Tiefeninput als Rohdaten

In [13]:
# Marker
example_counter = 5 # bis max 9

path_train_output = f"/home/se3_tracknet/train_output2"
path_train_output = f"/home/se3_tracknet/train_output"

file_dataset_info = f"{path_train_output}/dataset_info.yml"
file_images_mean = f"{path_train_output}/mean.npy"
file_images_std = f"{path_train_output}/std.npy"
file_ckpt_dir = f"{path_train_output}/model_best_val.pth.tar"
trans_normalizer = 0.02
rot_normalizer = 15

with open(file_dataset_info,'r') as ff:
    dataset_info = yaml.safe_load(ff)

images_std = np.load(file_images_std)
images_mean = np.load(file_images_mean)
tracker = Tracker(dataset_info,images_mean,images_std,file_ckpt_dir,model_path=dataset_info['models'][0]['model_path'],trans_normalizer=trans_normalizer,rot_normalizer=rot_normalizer)

# for own marker
pose_file_path = f"/home/se3_tracknet/generated_data_pair/train_data_blender_DR/000000{example_counter}meta.npz"
#rgb_A_file_path = f"/home/se3_tracknet/generated_data_pair/train_data_blender_DR/0000000rgbA.png"
rgb_file = f"/home/se3_tracknet/generated_data/000000{example_counter}rgb.png"
depth_file = f"/home/se3_tracknet/generated_data/000000{example_counter}depth.png"

pose_file = np.load(pose_file_path)
pose_a = pose_file['A_in_cam']

current_rgb = cv2.imread(rgb_file)
current_depth = cv2.imread(depth_file, cv2.IMREAD_GRAYSCALE).astype(np.uint16)
current_depth = fill_depth(current_depth/1e3,max_depth=2.0,extrapolate=False)
current_depth = (current_depth*1000).astype(np.uint16)

pose_b = tracker.on_track(pose_a,current_rgb.astype(np.uint8),current_depth)

print(pose_b)
print(pose_file['A_in_cam'])

self.object_cloud loaded and downsampled
self.object_width= 148.76080761074135
Loading ckpt from  /home/se3_tracknet/train_output/model_best_val.pth.tar
vorher_1:
Allocated: 105.1064453125 MB
Cached: 124.0 MB
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  107629 KB |  107629 KB |  269072 KB |  161443 KB |
|       from large pool |  105472 KB |  105472 KB |  263680 KB |  158208 KB |
|       from small pool |    2157 KB |    2157 KB |    5392 KB |    3235 KB |
|----------

FileNotFoundError: [Errno 2] No such file or directory: '/home/se3_tracknet/generated_data_pair/train_data_blender_DR/0000005meta.npz'

In [3]:
# properties for YCB
example_counter = 0 # bis max 9
base_dataset = f"/DATASET/SE3-TrackNet/YCB/YCB_traindata/large_marker"
base_dataset_traindata = f"{base_dataset}/train_data_blender_DR"
base_ckpt = f"/DATASET/SE3-TrackNet/YCB/YCB_weights/YCB_weights/large_marker"
pose_file_path = f"{base_dataset_traindata}/{example_counter:07}meta.npz"
rgb_A_file_path = f"{base_dataset_traindata}/{example_counter:07}rgbA.png"
rgb_B_file_path = f"{base_dataset_traindata}/{example_counter:07}rgbB.png"
depth_A_file_path = f"{base_dataset_traindata}/{example_counter:07}depthA.png"
depth_B_file_path = f"{base_dataset_traindata}/{example_counter:07}depthB.png"
file_images_mean = f"{base_ckpt}/mean.npy"
file_images_std = f"{base_ckpt}/std.npy"
file_dataset_info = f"{base_dataset}/dataset_info.yml"
file_ckpt_dir = f"{base_ckpt}/model_epoch240.pth.tar"
model_path = f"/DATASET/SE3-TrackNet/YCB/YCB_models_with_ply/CADmodels/040_large_marker/textured.ply"

# load data
rgbA = cv2.imread(rgb_A_file_path)
rgbB = cv2.imread(rgb_B_file_path)
depthA = cv2.imread(depth_A_file_path, cv2.IMREAD_GRAYSCALE).astype(np.uint16)
depthA = fill_depth(depthA/1e3,max_depth=2.0,extrapolate=False)
depthA = (depthA*1000).astype(np.uint16)
depthB = cv2.imread(depth_B_file_path, cv2.IMREAD_GRAYSCALE).astype(np.uint16)
depthB = fill_depth(depthB/1e3,max_depth=2.0,extrapolate=False)
depthB = (depthB*1000).astype(np.uint16)

pose_file = np.load(pose_file_path)
sample_poses = pose_file['A_in_cam']

images_mean = np.load(file_images_mean)
images_std = np.load(file_images_std)
posttransforms = Compose([OffsetDepth(),NormalizeChannels(images_mean, images_std),ToTensor()])
with open(file_dataset_info,'r') as ff:
    dataset_info = yaml.safe_load(ff)
trans_normalizer = dataset_info['max_translation']
rot_normalizer = dataset_info['max_rotation']
dataset = TrackDataset('','eval',images_mean, images_std,None,None,posttransforms,dataset_info, trans_normalizer=trans_normalizer, rot_normalizer=rot_normalizer)

# init model
image_size = (dataset_info['resolution'], dataset_info['resolution'])
checkpoint = torch.load(file_ckpt_dir)
model = Se3TrackNet(image_size=image_size[0])
model.load_state_dict(checkpoint['state_dict'])
model = model.cuda()
model.eval()

# preprocess data
dataAs = []
dataBs = []
sample, (trans_label, rot_label), rgbA_viz, rgbB_viz, maskA, maskB = dataset.processData(rgbA,depthA,sample_poses,rgbB,depthB,np.eye(4))
dataAs.append(sample[0].unsqueeze(0))
dataBs.append(sample[1].unsqueeze(0))

dataA = torch.cat(dataAs,dim=0).cuda().float()
dataB = torch.cat(dataBs,dim=0).cuda().float()

# prediction
with torch.no_grad():
      prediction = model(dataA,dataB)

# postprocess data
trans_pred = prediction['trans'][0].data.cpu().numpy()
rot_pred = prediction['rot'][0].data.cpu().numpy()
pred_B_in_cam = dataset.processPredict(sample_poses,(trans_pred,rot_pred))

# output data
print("Position A:")
print(pose_file['A_in_cam'])
print("Position B")
print(np.array(pose_file['B_in_cam']))
print("Prediction")
print(pred_B_in_cam)
loss = nn.MSELoss()
print(np.array(pose_file['B_in_cam'])[:3,3])
trans_loss = loss(torch.tensor(pred_B_in_cam[:3,3]),torch.tensor(pose_file['B_in_cam'][:3,3]))
rot_loss = loss(torch.tensor(pred_B_in_cam[:3,:3]),torch.tensor(pose_file['B_in_cam'][:3,:3]))
total_loss = loss(torch.tensor(pred_B_in_cam),torch.tensor(pose_file['B_in_cam']))
loss_sum = trans_loss + rot_loss
print(f"loss: \ntranslation:{trans_loss}, rotation:{rot_loss}")
print(f"total:{total_loss}, sum:{loss_sum}")


# init Tracker
trans_normalizer = 0.02
rot_normalizer = 15
tracker = Tracker(dataset_info,images_mean,images_std,file_ckpt_dir,model_path=model_path,trans_normalizer=trans_normalizer,rot_normalizer=rot_normalizer)

#visualization
rendered_pose_A, rendered_pose_A_depth = tracker.render_window(np.array(pose_file['A_in_cam'])) 
rendered_pose_B, rendered_pose_B_depth = tracker.render_window(np.array(pose_file['B_in_cam'])) 
rendered_estimation, rendered_estimation_depth = tracker.render_window(np.array(pred_B_in_cam))
cv2.putText(rgbA, "rgb A", (0,10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255,255,255))
cv2.putText(rgbB, "rgb B", (0,10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (100,255,100))
rendered_pose_A = cv2.putText(rendered_pose_A.copy(), "rendered pose A", (0,10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (100,255,100))
rendered_pose_B = cv2.putText(rendered_pose_B.copy(), "rendered pose B", (0,10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (100,255,100))
rendered_estimation = cv2.putText(rendered_estimation.copy(), "rendered Estimation", (0,10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (10,255,100))
canvas = makeCanvas([rgbA, rgbB, rendered_pose_A, rendered_pose_B, rendered_estimation], flipBR=False)
cv2.imshow('YCB Visualization',canvas)
cv2.waitKey(2)
print("END")

#crop Box
#bb = compute_bbox(pose_file['A_in_cam'], tracker.K, tracker.object_width/1000)
#print(bb)
#rendered_pose_A_crop, rendered_pose_A_crop = crop_bbox(rendered_pose_A, current_depth, bb, tracker.image_size)
#canvas = makeCanvas([rendered_pose_A_crop, rendered_pose_A_crop], flipBR=False)
#cv2.imshow('YCB Crops',canvas)
#cv2.waitKey(1)

self.cam_K:
 [[1.066778e+03 0.000000e+00 3.129869e+02]
 [0.000000e+00 1.067487e+03 2.413109e+02]
 [0.000000e+00 0.000000e+00 1.000000e+00]]
making dataset... for eval
#dataset: 0
self.trans_normalizer=0.02, self.rot_normalizer=15
Position A:
[[ 0.64246942  0.30275253 -0.70397014 -0.00825192]
 [-0.60190822 -0.36919033 -0.70809956 -0.00407853]
 [-0.4742779   0.87865777 -0.05496426  1.0264451 ]
 [ 0.          0.          0.          1.        ]]
Position B
[[ 6.42083777e-01  2.99020325e-01 -7.05914445e-01  2.46762875e-08]
 [-6.01294668e-01 -3.74782211e-01 -7.05679069e-01 -1.93370039e-08]
 [-4.75576571e-01  8.77567775e-01 -6.08422538e-02  1.02883472e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Prediction
[[ 0.6647541   0.25164192 -0.70340482 -0.00856909]
 [-0.59834297 -0.38442853 -0.70299389 -0.00374778]
 [-0.4473116   0.88819542 -0.10498227  1.02851244]
 [ 0.          0.          0.          1.        ]]
[ 2.46762875e-08 -1.93370039e-08  1.02883472e+00]
loss: 


/home/se3_tracknet/vispy_renderer.py:114: RuntimeWarning: invalid value encountered in true_divide
  vertex_normal = vertex_normal/np.linalg.norm(vertex_normal, axis=1).reshape(-1,1)


self.cam_K:
 [[1.066778e+03 0.000000e+00 3.129869e+02]
 [0.000000e+00 1.067487e+03 2.413109e+02]
 [0.000000e+00 0.000000e+00 1.000000e+00]]
making dataset... for eval
#dataset: 0
self.trans_normalizer=0.02, self.rot_normalizer=15
Vispy render
[[235 168]
 [235 306]
 [374 168]
 [374 306]]
Vispy render
[[244 172]
 [244 311]
 [382 172]
 [382 311]]
Vispy render
[[235 168]
 [235 307]
 [373 168]
 [373 307]]
END


## mehrfaches Tracking auf einem Frame

### generated Dataset for se3_tracknet

In [15]:
# YCB Dataset large Marker
example_counter = 3 # bis max 9
FRAME_COUNT = 8

base_dataset = f"/DATASET/SE3-TrackNet/YCB/YCB_traindata/large_marker"
base_dataset_traindata = f"{base_dataset}/train_data_blender_DR"
base_ckpt = f"/DATASET/SE3-TrackNet/YCB/YCB_weights/YCB_weights/large_marker"
pose_file_path = f"{base_dataset_traindata}/{example_counter:07}meta.npz"
rgb_A_file_path = f"{base_dataset_traindata}/{example_counter:07}rgbA.png"
rgb_B_file_path = f"{base_dataset_traindata}/{example_counter:07}rgbB.png"
depth_A_file_path = f"{base_dataset_traindata}/{example_counter:07}depthA.png"
depth_B_file_path = f"{base_dataset_traindata}/{example_counter:07}depthB.png"
file_images_mean = f"{base_ckpt}/mean.npy"
file_images_std = f"{base_ckpt}/std.npy"
file_dataset_info = f"{base_dataset}/dataset_info.yml"
file_ckpt_dir = f"{base_ckpt}/model_epoch240.pth.tar"
model_path = f"/DATASET/SE3-TrackNet/YCB/YCB_models_with_ply/CADmodels/040_large_marker/textured.ply"

In [6]:
# YCB Dataset bleach cleanser
example_counter = 3 # bis max 9
FRAME_COUNT = 100

base_dataset = f"/DATASET/SE3-TrackNet/YCB/YCB_traindata/bleach_cleanser"
base_dataset_traindata = f"{base_dataset}/train_data_blender_DR"
base_ckpt = f"/DATASET/SE3-TrackNet/YCB/YCB_weights/YCB_weights/bleach_cleanser"
pose_file_path = f"{base_dataset_traindata}/{example_counter:07}meta.npz"
rgb_A_file_path = f"{base_dataset_traindata}/{example_counter:07}rgbA.png"
rgb_B_file_path = f"{base_dataset_traindata}/{example_counter:07}rgbB.png"
depth_A_file_path = f"{base_dataset_traindata}/{example_counter:07}depthA.png"
depth_B_file_path = f"{base_dataset_traindata}/{example_counter:07}depthB.png"
file_images_mean = f"{base_ckpt}/mean.npy"
file_images_std = f"{base_ckpt}/std.npy"
file_dataset_info = f"{base_dataset}/dataset_info.yml"
file_ckpt_dir = f"{base_ckpt}/model_best_val.pth.tar"
model_path = f"/DATASET/SE3-TrackNet/YCB/YCB_models_with_ply/CADmodels/021_bleach_cleanser/textured.ply"

In [9]:
# properties for own Marker
example_counter = 5 # bis max 9
FRAME_COUNT = 200

base_dataset = f"/home/se3_tracknet/generated_data_pair"
base_dataset_traindata = f"{base_dataset}/train_data_blender_DR"
base_ckpt = f"/home/se3_tracknet/train_output4"
pose_file_path = f"{base_dataset_traindata}/{example_counter:07}meta.npz"
rgb_A_file_path = f"{base_dataset_traindata}/{example_counter:07}rgbA.png"
rgb_B_file_path = f"{base_dataset_traindata}/{example_counter:07}rgbB.png"
depth_A_file_path = f"{base_dataset_traindata}/{example_counter:07}depthA.png"
depth_B_file_path = f"{base_dataset_traindata}/{example_counter:07}depthB.png"
file_images_mean = f"{base_ckpt}/mean.npy"
file_images_std = f"{base_ckpt}/std.npy"
file_dataset_info = f"{base_ckpt}/dataset_info.yml"
file_ckpt_dir = f"{base_ckpt}/model_best_val.pth.tar"
model_path = f"/DATASET/models/Marker/textured.obj"

In [36]:
# properties for own Marker with trained large marker from ycb
example_counter = 5 # bis max 9
FRAME_COUNT = 200
base_dataset = f"/home/se3_tracknet/generated_data_pair"
base_dataset_traindata = f"{base_dataset}/train_data_blender_DR"
base_ckpt = f"/DATASET/SE3-TrackNet/YCB/YCB_weights/YCB_weights/large_marker"

pose_file_path = f"{base_dataset_traindata}/{example_counter:07}meta.npz"
rgb_A_file_path = f"{base_dataset_traindata}/{example_counter:07}rgbA.png"
rgb_B_file_path = f"{base_dataset_traindata}/{example_counter:07}rgbB.png"
depth_A_file_path = f"{base_dataset_traindata}/{example_counter:07}depthA.png"
depth_B_file_path = f"{base_dataset_traindata}/{example_counter:07}depthB.png"

file_images_mean = f"{base_ckpt}/mean.npy"
file_images_std = f"{base_ckpt}/std.npy"
file_dataset_info = f"{base_dataset}/dataset_info.yml"
file_ckpt_dir = f"{base_ckpt}/model_epoch240.pth.tar"
model_path = f"/DATASET/SE3-TrackNet/YCB/YCB_models_with_ply/CADmodels/040_large_marker/textured.ply"

In [10]:
# trained model
example_counter = 2 # bis max 9
FRAME_COUNT = 50
base_dataset = f"/home/se3_tracknet/generated_Datasets/002/generated_data_pair"
base_dataset_traindata = f"{base_dataset}/train_data_blender_DR"
base_ckpt = f"/home/se3_tracknet/train_output"

pose_file_path = f"{base_dataset_traindata}/{example_counter:07}meta.npz"
rgb_A_file_path = f"{base_dataset_traindata}/{example_counter:07}rgbA.png"
rgb_B_file_path = f"{base_dataset_traindata}/{example_counter:07}rgbB.png"
depth_A_file_path = f"{base_dataset_traindata}/{example_counter:07}depthA.png"
depth_B_file_path = f"{base_dataset_traindata}/{example_counter:07}depthB.png"

file_images_mean = f"{base_ckpt}/mean.npy"
file_images_std = f"{base_ckpt}/std.npy"
file_dataset_info = f"{base_dataset}/dataset_info.yml"
file_ckpt_dir = f"{base_ckpt}/model_best_val.pth.tar"
model_path = f"/DATASET/models/Pferd/textured.obj"

In [11]:
# load data
rgbA = cv2.imread(rgb_A_file_path)
rgbB = cv2.imread(rgb_B_file_path)
depthA = cv2.imread(depth_A_file_path, cv2.IMREAD_GRAYSCALE).astype(np.uint16)
depthA = fill_depth(depthA/1e3,max_depth=2.0,extrapolate=False)
depthA = (depthA*1000).astype(np.uint16)
depthB = cv2.imread(depth_B_file_path, cv2.IMREAD_GRAYSCALE).astype(np.uint16)
depthB = fill_depth(depthB/1e3,max_depth=2.0,extrapolate=False)
depthB = (depthB*1000).astype(np.uint16)

pose_file = np.load(pose_file_path)
sample_poses = pose_file['A_in_cam']

images_mean = np.load(file_images_mean)
images_std = np.load(file_images_std)
posttransforms = Compose([OffsetDepth(),NormalizeChannels(images_mean, images_std),ToTensor()])
with open(file_dataset_info,'r') as ff:
    dataset_info = yaml.safe_load(ff)
trans_normalizer = dataset_info['max_translation']
rot_normalizer = dataset_info['max_rotation']
dataset = TrackDataset('','eval',images_mean, images_std,None,None,posttransforms,dataset_info)

# init model
image_size = (dataset_info['resolution'], dataset_info['resolution'])
checkpoint = torch.load(file_ckpt_dir)
model = Se3TrackNet(image_size=image_size[0])
model.load_state_dict(checkpoint['state_dict'])
model = model.cuda()
model.eval()

# init Tracker (for visualization)
trans_normalizer = 0.02
rot_normalizer = 15

tracker = Tracker(dataset_info,images_mean,images_std,file_ckpt_dir,model_path=model_path,trans_normalizer=trans_normalizer,rot_normalizer=rot_normalizer)


rendered_poses = []
poses = []
rendered_poses.append(rgbB)
rendered_pose_A, rendered_pose_A_depth = tracker.render_window(np.array(pose_file['A_in_cam'])) 
current_pose = np.array(pose_file['A_in_cam']).copy()

np.set_printoptions(suppress=True, precision=3)
for i in range(FRAME_COUNT):
    print(f"Frame {i}")
    # preprocess data
    dataAs = []
    dataBs = []
    rendered_pose, rendered_pose_depth = tracker.render_window(current_pose)
    rendered_poses.append(rendered_pose.copy())
    poses.append(current_pose)
    sample, (trans_label, rot_label), rgbA_viz, rgbB_viz, maskA, maskB = dataset.processData(rendered_pose,rendered_pose_depth,current_pose.copy(),rgbB,depthB,np.eye(4))
    dataAs.append(sample[0].unsqueeze(0))
    dataBs.append(sample[1].unsqueeze(0))
    
    dataA = torch.cat(dataAs,dim=0).cuda().float()
    dataB = torch.cat(dataBs,dim=0).cuda().float()
    
    # prediction
    with torch.no_grad():
          prediction = model(dataA,dataB)
    
    # postprocess data
    trans_pred = prediction['trans'][0].data.cpu().numpy()
    rot_pred = prediction['rot'][0].data.cpu().numpy()
    prediction_position = dataset.processPredict(sample_poses,(trans_pred,rot_pred))
    
    # output data
    loss = nn.MSELoss()
    trans_loss = loss(torch.tensor(prediction_position[:3,3]),torch.tensor(pose_file['B_in_cam'][:3,3]))
    rot_loss = loss(torch.tensor(prediction_position[:3,:3]),torch.tensor(pose_file['B_in_cam'][:3,:3]))
    total_loss = loss(torch.tensor(prediction_position),torch.tensor(pose_file['B_in_cam']))
    loss_sum = trans_loss + rot_loss
    print(f"translation_loss:{trans_loss:.7}, rotation_loss:{rot_loss:.7}, total_loss:{total_loss:.7}, sum_loss:{loss_sum:.7}")

    rendered_prediction, rendered_prediction_depth = tracker.render_window(prediction_position)
    
    stats_real = np.zeros(rendered_prediction.shape)
    stats_prediction = np.zeros(rendered_prediction.shape)
    stats_prediction = cv2.putText(stats_prediction, f"Translation:", (0,10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"{prediction_position[:3,3]}", (0,30), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"T-loss:{float(trans_loss):.7f}", (0,50), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"Rotation:", (0,70), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"{prediction_position[0,:3]}", (0,90), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"{prediction_position[1,:3]}", (0,110), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"{prediction_position[2,:3]}", (0,130), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"loss:{rot_loss:.4}", (0,150), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"total-loss:{loss_sum:.5} ", (0,170), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    
    stats_real = cv2.putText(stats_real, f"Translation:", (0,10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_real = cv2.putText(stats_real, f"{pose_file['B_in_cam'][:3,3]}", (0,30), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_real = cv2.putText(stats_real, f"Rotation:", (0,70), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_real = cv2.putText(stats_real, f"{pose_file['B_in_cam'][0,:3]}", (0,90), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_real = cv2.putText(stats_real, f"{pose_file['B_in_cam'][1,:3]}", (0,110), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_real = cv2.putText(stats_real, f"{pose_file['B_in_cam'][2,:3]}", (0,130), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))

    canvas = makeCanvas([rgbB,rendered_pose,rendered_prediction,stats_prediction,stats_real], flipBR=False)
    cv2.imshow("multi tracking on Frame 1", canvas)
    cv2.waitKey(0)

    current_pose = prediction_position
cv2.destroyAllWindows()
#poses.append(current_pose)
#rendered_pose, rendered_pose_depth = tracker.render_window(current_pose)
#rendered_poses.append(rendered_pose.copy())

#for i in range(FRAME_COUNT):
#    np.set_printoptions(suppress=True, precision=8)
#    print(f"{poses[i][:3,3]}")

#visualization
#canvas = makeCanvas(rendered_poses, flipBR=False)
#cv2.imshow('YCB multi tracking on Frame',canvas)
#cv2.waitKey(2)
#print("END")

self.cam_K:
 [[900.   0. 640.]
 [  0. 900. 360.]
 [  0.   0.   1.]]
making dataset... for eval
#dataset: 0
self.trans_normalizer=0.03, self.rot_normalizer=0.08726646259971647
self.object_cloud loaded and downsampled
self.object_width= 163.63688837181547
Loading ckpt from  /home/se3_tracknet/train_output/model_best_val.pth.tar
vorher_1:
Allocated: 212.3818359375 MB
Cached: 242.0 MB
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  217479 KB |  217479 KB |   23223 MB |   230

### Raw Input Data
On Track function

In [26]:
# config own marker
# Marker and current output
FRAME_COUNT = 100
example_counter = 6

#path_train_output = f"/home/se3_tracknet/train_output2" #  Marker trained
path_train_output = f"/home/se3_tracknet/train_output"

file_dataset_info = f"{path_train_output}/dataset_info.yml"
file_images_mean = f"{path_train_output}/mean.npy"
file_images_std = f"{path_train_output}/std.npy"
file_ckpt_dir = f"{path_train_output}/model_best_val.pth.tar"

# for own marker
pose_file_path = f"/home/se3_tracknet/generated_data_pair/train_data_blender_DR/{example_counter:07}meta.npz"
#rgb_A_file_path = f"/home/se3_tracknet/generated_data_pair/train_data_blender_DR/0000000rgbA.png"
rgb_file = f"/home/se3_tracknet/generated_data/{example_counter:07}rgb.png"
depth_file = f"/home/se3_tracknet/generated_data/{example_counter:07}depth.png"

pose_file = np.load(pose_file_path)
pose_a = pose_file['A_in_cam']
pose_b = pose_file['B_in_cam']

with open(file_dataset_info,'r') as ff:
    dataset_info = yaml.safe_load(ff)
    
model_path = dataset_info['models'][0]['model_path']

current_rgb = cv2.imread(rgb_file)
current_depth = cv2.imread(depth_file, cv2.IMREAD_GRAYSCALE).astype(np.uint16)
current_depth = fill_depth(current_depth/1e3,max_depth=2.0,extrapolate=False)
current_depth = (current_depth*1000).astype(np.uint16)

FileNotFoundError: [Errno 2] No such file or directory: '/home/se3_tracknet/generated_data_pair/train_data_blender_DR/0000006meta.npz'

In [29]:
# config mix (own marker with trained YCB-model)
# config own marker
# Marker and current output
FRAME_COUNT = 50
example_counter = 3

#path_train_output = f"/home/se3_tracknet/train_output2" #  Marker trained
path_train_output = f"/home/se3_tracknet/train_output2"

file_dataset_info = f"{path_train_output}/dataset_info.yml"
file_images_mean = f"{path_train_output}/mean.npy"
file_images_std = f"{path_train_output}/std.npy"

base_ckpt = f"/DATASET/SE3-TrackNet/YCB/YCB_weights/YCB_weights/large_marker"
file_ckpt_dir = f"{base_ckpt}/model_epoch240.pth.tar"

pose_file_path = f"/home/se3_tracknet/generated_data_pair/train_data_blender_DR/{example_counter:07}meta.npz"
rgb_file = f"/home/se3_tracknet/generated_data/{example_counter:07}rgb.png"
depth_file = f"/home/se3_tracknet/generated_data/{example_counter:07}depth.png"


pose_file = np.load(pose_file_path)
pose_a = pose_file['A_in_cam']
pose_b = pose_file['B_in_cam']

with open(file_dataset_info,'r') as ff:
    dataset_info = yaml.safe_load(ff)
    
model_path = dataset_info['models'][0]['model_path']

current_rgb = cv2.imread(rgb_file)
current_depth = cv2.imread(depth_file, cv2.IMREAD_GRAYSCALE).astype(np.uint16)
current_depth = fill_depth(current_depth/1e3,max_depth=2.0,extrapolate=False)
current_depth = (current_depth*1000).astype(np.uint16)

In [23]:
# config YCB-model Marker
# Marker and current output
FRAME_COUNT = 50
example_counter = 6 
example_counter += 1
obj_id = 18

base_dataset = f"/DATASET/SE3-TrackNet/YCB/YCB_traindata/large_marker"
file_dataset_info = f"{base_dataset}/dataset_info.yml"

base_ckpt = f"/DATASET/SE3-TrackNet/YCB/YCB_weights/YCB_weights/large_marker"
file_ckpt_dir = f"{base_ckpt}/model_epoch240.pth.tar"
file_images_mean = f"{base_ckpt}/mean.npy"
file_images_std = f"{base_ckpt}/std.npy"

ycb_data_path = f"/DATASET/SE3-TrackNet/YCB/YCB_reorganized/0057"
ycb_data_rgb_path = f"{ycb_data_path}/color"
ycb_data_depth_path = f"{ycb_data_path}/depth"
ycb_data_pose_path = f"{ycb_data_path}/pose_gt/{obj_id}"

pose_a_file_path = f"{ycb_data_pose_path}/{(example_counter-1):06}.txt"
pose_b_file_path = f"{ycb_data_pose_path}/{example_counter:06}.txt"

rgb_file = f"{ycb_data_rgb_path}/{example_counter:06}.png"
depth_file = f"{ycb_data_depth_path}/{example_counter:06}.png"



pose_a = np.loadtxt(pose_a_file_path, delimiter=' ')
pose_b = np.loadtxt(pose_b_file_path, delimiter=' ')

model_path = f"/DATASET/SE3-TrackNet/YCB/YCB_models_with_ply/CADmodels/040_large_marker/textured.ply"

current_rgb = cv2.imread(rgb_file,cv2.IMREAD_COLOR)
current_depth = cv2.imread(depth_file, cv2.IMREAD_UNCHANGED).astype(np.uint16)

In [21]:
# config YCB-model bleach clenser
# Marker and current output
FRAME_COUNT = 50
example_counter = 6 
example_counter += 1
obj_id = 12

base_dataset = f"/DATASET/SE3-TrackNet/YCB/YCB_traindata/bleach_cleanser"
file_dataset_info = f"{base_dataset}/dataset_info.yml"

base_ckpt = f"/DATASET/SE3-TrackNet/YCB/YCB_weights/YCB_weights/bleach_cleanser"
file_ckpt_dir = f"{base_ckpt}/model_best_val.pth.tar"
file_images_mean = f"{base_ckpt}/mean.npy"
file_images_std = f"{base_ckpt}/std.npy"

ycb_data_path = f"/DATASET/SE3-TrackNet/YCB/YCB_reorganized/0051"
ycb_data_rgb_path = f"{ycb_data_path}/color"
ycb_data_depth_path = f"{ycb_data_path}/depth"
ycb_data_pose_path = f"{ycb_data_path}/pose_gt/{obj_id}"

pose_a_file_path = f"{ycb_data_pose_path}/{(example_counter-1):06}.txt"
pose_b_file_path = f"{ycb_data_pose_path}/{example_counter:06}.txt"

rgb_file = f"{ycb_data_rgb_path}/{example_counter:06}.png"
depth_file = f"{ycb_data_depth_path}/{example_counter:06}.png"



pose_a = np.loadtxt(pose_a_file_path, delimiter=' ')
pose_b = np.loadtxt(pose_b_file_path, delimiter=' ')

model_path = f"/DATASET/SE3-TrackNet/YCB/YCB_models_with_ply/CADmodels/021_bleach_cleanser/textured.ply"

current_rgb = cv2.imread(rgb_file,cv2.IMREAD_COLOR)
current_depth = cv2.imread(depth_file, cv2.IMREAD_UNCHANGED).astype(np.uint16)

In [4]:
with open(file_dataset_info,'r') as ff:
    dataset_info = yaml.safe_load(ff)

images_std = np.load(file_images_std)
images_mean = np.load(file_images_mean)
tracker = Tracker(dataset_info,images_mean,images_std,file_ckpt_dir,model_path=model_path)

rendered_poses = []
poses = []
rendered_pose_A, rendered_pose_A_depth = tracker.render_window(np.array(pose_a)) 
current_pose = np.array(pose_a).copy()

for i in range(FRAME_COUNT):
    print(f"Frame {i}")

    rendered_pose, rendered_pose_depth = tracker.render_window(current_pose)
    rendered_poses.append(rendered_pose.copy())
    poses.append(current_pose)

    prediction_pose = tracker.on_track(current_pose,current_rgb.astype(np.uint8),current_depth)
    
    # output loss
    loss = nn.MSELoss()
    trans_loss = loss(torch.tensor(prediction_pose[:3,3]),torch.tensor(pose_b[:3,3]))
    rot_loss = loss(torch.tensor(prediction_pose[:3,:3]),torch.tensor(pose_b[:3,:3]))
    loss_sum = rot_loss + trans_loss
    print(f"translation_loss:{trans_loss:.7}, rotation_loss:{rot_loss:.7}")

    rendered_prediction, rendered_prediction_depth = tracker.render_window(prediction_pose)
    np.set_printoptions(suppress=True, precision=3)
    stats_real = np.zeros(rendered_prediction.shape)
    stats_prediction = np.zeros(rendered_prediction.shape)
    stats_prediction = cv2.putText(stats_prediction, f"Translation:", (0,10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"{prediction_pose[:3,3]}", (0,30), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"T-loss:{float(trans_loss):.7f}", (0,50), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"Rotation:", (0,70), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"{prediction_pose[0,:3]}", (0,90), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"{prediction_pose[1,:3]}", (0,110), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"{prediction_pose[2,:3]}", (0,130), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"loss:{rot_loss:.4}", (0,150), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"total-loss:{loss_sum:.5} ", (0,170), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    
    stats_real = cv2.putText(stats_real, f"Translation:", (0,10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_real = cv2.putText(stats_real, f"{pose_b[:3,3]}", (0,30), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_real = cv2.putText(stats_real, f"Rotation:", (0,70), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_real = cv2.putText(stats_real, f"{pose_b[0,:3]}", (0,90), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_real = cv2.putText(stats_real, f"{pose_b[1,:3]}", (0,110), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_real = cv2.putText(stats_real, f"{pose_b[2,:3]}", (0,130), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))

    canvas = makeCanvas([rendered_pose,rendered_prediction,stats_prediction,stats_real], flipBR=False)
    cv2.imshow("multi tracking on Frame", canvas)
    if i<2:
        cv2.waitKey(0)
    
    current_pose = prediction_pose

cv2.destroyAllWindows()
print("END")

self.object_cloud loaded and downsampled
self.object_width= 148.76080761074135
Loading ckpt from  /home/se3_tracknet/train_output2/model_best_val.pth.tar
net init done
Using vispy renderer
model_path:  /DATASET/SE3-TrackNet/YCB/YCB_models_with_ply/CADmodels/021_bleach_cleanser/textured.ply
self.cam_K:
 [[900.   0. 640.]
 [  0. 900. 360.]
 [  0.   0.   1.]]
making dataset... for eval
#dataset: 0
self.trans_normalizer=0.03, self.rot_normalizer=0.08726646259971647
Frame 0
translation_loss:6.235359e-05, rotation_loss:0.01655389
Frame 1
translation_loss:0.0003120189, rotation_loss:0.01957697
Frame 2
translation_loss:0.0007403491, rotation_loss:0.02334021
Frame 3
translation_loss:0.001273135, rotation_loss:0.02655055
Frame 4
translation_loss:0.00192478, rotation_loss:0.03012223
Frame 5
translation_loss:0.002666164, rotation_loss:0.0345155
Frame 6
translation_loss:0.00348994, rotation_loss:0.03753421
Frame 7
translation_loss:0.00432379, rotation_loss:0.03701697
Frame 8
translation_loss:0.0051

KeyboardInterrupt: 

# Tracking on multiple Frames
(YCB -Video Dataset)

In [8]:
# Marker and current output
FRAME_COUNT = 20
example_counter = 4
obj_id = 12

#path_train_output = f"/home/se3_tracknet/train_output2" #  Marker trained
path_train_output = f"/DATASET/SE3-TrackNet/YCB/YCB_weights/YCB_weights/bleach_cleanser"
ycb_data_path = f"/DATASET/SE3-TrackNet/YCB/YCB_reorganized/0051"
ycb_data_rgb_path = f"{ycb_data_path}/color"
ycb_data_depth_path = f"{ycb_data_path}/depth"

poses_file_path = f"{ycb_data_path}/pose_gt/{obj_id}"

file_dataset_info = f"/DATASET/SE3-TrackNet/YCB/YCB_traindata/bleach_cleanser/dataset_info.yml"
file_dataset_info = f"{ycb_data_path}/dataset_info.yml"

file_images_mean = f"{path_train_output}/mean.npy"
file_images_std = f"{path_train_output}/std.npy"
file_ckpt_dir = f"{path_train_output}/model_best_val.pth.tar"
model_path = f"/DATASET/SE3-TrackNet/YCB/YCB_models_with_ply/CADmodels/021_bleach_cleanser/textured.ply"

trans_normalizer = 0.02
rot_normalizer = 15


with open(file_dataset_info,'r') as ff:
    dataset_info = yaml.safe_load(ff)

images_std = np.load(file_images_std)
images_mean = np.load(file_images_mean)
tracker = Tracker(dataset_info,images_mean,images_std,file_ckpt_dir,model_path=model_path)

rendered_poses = []
poses = []

current_pose = np.loadtxt(f"{poses_file_path}/000000.txt", delimiter=' ')

for i in range(1,FRAME_COUNT):
    print(f"Frame {i}")

    rendered_pose, rendered_pose_depth = tracker.render_window(current_pose)
    rendered_poses.append(rendered_pose.copy())
    poses.append(current_pose)

    current_rgb_path = f"{ycb_data_rgb_path}/{i:06}.png"
    current_depth_path = f"{ycb_data_depth_path}/{i:06}.png"

    current_rgb = cv2.imread(current_rgb_path,cv2.IMREAD_COLOR)
    #current_rgb = np.array(Image.open(current_rgb_path))
    current_depth = cv2.imread(current_depth_path, cv2.IMREAD_UNCHANGED).astype(np.uint16)
    
    # get reference pose
    ref_pose = np.loadtxt(f"{poses_file_path}/{i:06}.txt", delimiter=' ')
    prev_pose = np.loadtxt(f"{poses_file_path}/{(i-1):06}.txt", delimiter=' ')
    
    prediction_pose = tracker.on_track(current_pose,current_rgb.astype(np.uint8),current_depth)

    
    # output loss
    loss = nn.MSELoss()
    trans_loss = loss(torch.tensor(prediction_pose[:3,3]),torch.tensor(ref_pose[:3,3]))
    rot_loss = loss(torch.tensor(prediction_pose[:3,:3]),torch.tensor(ref_pose[:3,:3]))
    loss_sum = rot_loss + trans_loss
    print(f"translation_loss:{trans_loss:.7}, rotation_loss:{rot_loss:.7}")

    rendered_prediction, rendered_prediction_depth = tracker.render_window(prediction_pose)
    np.set_printoptions(suppress=True, precision=3)
    stats_real = np.zeros(rendered_prediction.shape)
    stats_prediction = np.zeros(rendered_prediction.shape)
    stats_prediction = cv2.putText(stats_prediction, f"Translation:", (0,10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"{prediction_pose[:3,3]}", (0,30), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"T-loss:{float(trans_loss):.7f}", (0,50), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"Rotation:", (0,70), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"{prediction_pose[0,:3]}", (0,90), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"{prediction_pose[1,:3]}", (0,110), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"{prediction_pose[2,:3]}", (0,130), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"loss:{rot_loss:.4}", (0,150), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_prediction = cv2.putText(stats_prediction, f"total-loss:{loss_sum:.5} ", (0,170), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    
    stats_real = cv2.putText(stats_real, f"Translation:", (0,10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_real = cv2.putText(stats_real, f"{ref_pose[:3,3]}", (0,30), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_real = cv2.putText(stats_real, f"Rotation:", (0,70), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_real = cv2.putText(stats_real, f"{ref_pose[0,:3]}", (0,90), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_real = cv2.putText(stats_real, f"{ref_pose[1,:3]}", (0,110), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))
    stats_real = cv2.putText(stats_real, f"{ref_pose[2,:3]}", (0,130), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100,255,100))

    canvas = makeCanvas([rendered_pose,rendered_prediction,stats_prediction,stats_real], flipBR=False)
    cv2.imshow("multi tracking on Frame", canvas)

    #rendered object model
    model = copy.deepcopy(tracker.object_cloud)
    model.transform(prediction_pose)
    uvs = project_points(np.asarray(model.points).copy(),tracker.K.copy())
    cur_bgr = cv2.cvtColor(current_rgb.copy(),cv2.COLOR_RGB2BGR)
    for ii in range(len(uvs)):
        cv2.circle(cur_bgr,(uvs[ii,0],uvs[ii,1]),radius=1,color=(0,255,255),thickness=-1)
    cv2.imshow("rendered Model",cur_bgr)
    
    if i<5:
        cv2.waitKey(0)
    
    
    current_pose = prediction_pose

cv2.destroyAllWindows()

self.object_cloud loaded and downsampled
self.object_width= 285.37860394994397
Loading ckpt from  /DATASET/SE3-TrackNet/YCB/YCB_weights/YCB_weights/bleach_cleanser/model_best_val.pth.tar
pose track ckpt epoch=112
net init done
Using vispy renderer
model_path:  /DATASET/SE3-TrackNet/YCB/YCB_models_with_ply/CADmodels/021_bleach_cleanser/textured.ply
self.cam_K:
 [[1066.778    0.     312.987]
 [   0.    1067.487  241.311]
 [   0.       0.       1.   ]]
making dataset... for eval
#dataset: 0
self.trans_normalizer=0.03, self.rot_normalizer=0.08726646259971647
Frame 1
translation_loss:6.065442e-06, rotation_loss:4.465799e-05
Frame 2
translation_loss:2.9172e-06, rotation_loss:7.315967e-05
Frame 3
translation_loss:3.454821e-06, rotation_loss:0.0001189056
Frame 4
translation_loss:4.114732e-06, rotation_loss:0.0001666099
Frame 5
translation_loss:4.466842e-06, rotation_loss:0.0001620321
Frame 6
translation_loss:3.234038e-06, rotation_loss:0.0001889235
Frame 7
translation_loss:1.020229e-05, rotati

## Train Model
Code from train.py

In [3]:
use_pretrained_model = False
#pretrained_model_dkpt_path = f"/home/se3_tracknet/train_output2/model_best_val.pth.tar"
pretrained_model_dkpt_path = f"/DATASET/SE3-TrackNet/YCB/YCB_weights/YCB_weights/large_marker/model_epoch240.pth.tar"
dir_path = f"/home/se3_tracknet"
training_logs="/home/se3_tracknet/logs/run5"

torch.cuda.empty_cache()

with open(dir_path+'/config.yml', 'r') as ff:
	config = yaml.safe_load(ff)

sys.path.append(dir_path)

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.benchmark = True

data_path = config['data_path']
validation_path = config['validation_path']

output_path = f"{dir_path}/train_output/"

print('output_path',output_path)
if not os.path.exists(output_path):
	os.makedirs(output_path)
with open(data_path+'/../dataset_info.yml', 'r') as ff:
	dataset_info = yaml.safe_load(ff)
	print('loaded dataset info from:',data_path+'/../dataset_info.yml')
shutil.copy(data_path+'/../dataset_info.yml',f'{output_path}/dataset_info.yml')
hsv_noise = config['data_augmentation']['hsv_noise']
batch_size = config['batch_size']
n_workers = config['n_workers']
augmentations = Compose([
						HSVJitter(hsv_noise[0],hsv_noise[1],hsv_noise[2]),
						ChangeBright(prob=0.5,mag=[config['data_augmentation']['bright_mag'][0], config['data_augmentation']['bright_mag'][1]]),
						GaussianNoise(config['data_augmentation']['gaussian_noise']['rgb'], config['data_augmentation']['gaussian_noise']['depth']),
						GaussianBlur(config['data_augmentation']['gaussian_blur_kernel']), BlackCover(prob=0.2)])
############ Compute mean std
posttransforms = Compose([OffsetDepth(),Transpose(),ToTensor()])  #NOTE Transpose() make image CxHxW
train_dataset = TrackDataset(data_path,mode='train',images_mean=None,images_std=None,pretransforms=None, augmentations=augmentations, posttransforms=posttransforms, dataset_info=dataset_info, trans_normalizer=dataset_info['max_translation'], rot_normalizer=dataset_info['max_rotation']*np.pi/180)
print('len(train_dataset)=',len(train_dataset))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=n_workers, pin_memory=False, drop_last=True, )
n = 10000
print('Computing mean std for n={}'.format(n))
total = 0
image_means_array = []
for i, (data, target, A_in_cams, B_in_cams, rgbA, rgbB, maskA, maskB) in enumerate(train_loader):
	bufferA, bufferB = data
	bufferA_numpy = bufferA.cpu().numpy()[:,:4]
	bufferB_numpy = bufferB.cpu().numpy()[:,:4]
	buffer_numpy = np.concatenate((bufferA_numpy, bufferB_numpy), axis=1)  #NxCxHxW
	image_means = np.mean(buffer_numpy, axis=(0, 2, 3))
	image_means_array.append(image_means)
	total += 1
	if i * batch_size >= n:
			break
images_mean = np.mean(image_means_array, axis=0)
images_std = np.std(image_means_array, axis=0)
np.save("{}/mean.npy".format(output_path), images_mean)
np.save("{}/std.npy".format(output_path), images_std)
print('images_mean\n',images_mean)
print('images_std\n',images_std)
posttransforms = Compose([OffsetDepth(),NormalizeChannels(images_mean,images_std),ToTensor()])
train_dataset = TrackDataset(data_path,'train',images_mean,images_std,None,augmentations,posttransforms,dataset_info=dataset_info, trans_normalizer=dataset_info['max_translation'], rot_normalizer=dataset_info['max_rotation']*np.pi/180)

valid_dataset = TrackDataset(validation_path,'val',images_mean,images_std,None,augmentations,posttransforms,dataset_info=dataset_info, trans_normalizer=dataset_info['max_translation'], rot_normalizer=dataset_info['max_rotation']*np.pi/180)
with open(os.path.join(output_path, "config_backup.yml"), 'w') as ff:
	yaml.dump(config, ff)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=n_workers, pin_memory=False, drop_last=True, )
assert len(train_loader)>0
val_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=n_workers, pin_memory=False, drop_last=False, )
model = Se3TrackNet(image_size=int(dataset_info["resolution"]))
#loading a pretrained model
if use_pretrained_model:
    pretrained_model_ckpt = torch.load(pretrained_model_dkpt_path)
    model.load_state_dict(pretrained_model_ckpt["state_dict"])
model = model.cuda()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=config['learning_rate'], weight_decay=config['weight_decay'], betas=(0.9, 0.99), amsgrad=False)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,200,300], gamma=0.1)
problem = Problem(model, train_loader, val_loader,config=config,optimizer=optimizer, scheduler=scheduler,log_dir=training_logs)
print("Training Begins:")
problem.loop(config['epochs'], output_path, save_all_checkpoints=True,logs=True)
print("Training Complete")

output_path /home/se3_tracknet/train_output/
loaded dataset info from: /home/se3_tracknet/generated_Datasets/002/generated_data_pair/train_data_blender_DR/../dataset_info.yml
self.cam_K:
 [[900.   0. 640.]
 [  0. 900. 360.]
 [  0.   0.   1.]]
making dataset... for train
#dataset: 25665
self.trans_normalizer=0.05, self.rot_normalizer=0.5235987755982988
len(train_dataset)= 25665
Computing mean std for n=10000
images_mean
 [  16.892878     7.4138165    7.0143595 1671.8474     134.86937
  125.05047    121.757385   394.192    ]
images_std
 [ 0.45829082  0.330759    0.36450934  8.575679    5.640431    5.5281525
  5.6854978  37.497547  ]
self.cam_K:
 [[900.   0. 640.]
 [  0. 900. 360.]
 [  0.   0.   1.]]
making dataset... for train
#dataset: 25665
self.trans_normalizer=0.05, self.rot_normalizer=0.5235987755982988
self.cam_K:
 [[900.   0. 640.]
 [  0. 900. 360.]
 [  0.   0.   1.]]
making dataset... for val
#dataset: 2000
self.trans_normalizer=0.05, self.rot_normalizer=0.5235987755982988
Traini